Fazer um chatbot capaz de responder a perguntas sobre a declaração do imposto de renda usando sistema multiagente.
Requisitos:
* Usar pelo menos 2 agentes.
Sugestão: Agente buscador (do exercício anterior) + Agente redator (Definir humor: alegre, triste, grosso, etc.)
* Usar Langgraph, Autogen, Crew-AI ou outro framework multiagent
* Salvar memória (histórico de conversa)
* Ter interface (streamlit, gradio, etc.)

Avaliação: Como avaliar um chatbot com RAG?
Considerando que eu não tenho respostas de referência. Ou seja, sem dataset.


### Ler Dados

In [ ]:
import json

In [ ]:
!git clone https://huggingface.co/datasets/unicamp-dl/rag-rfb

Cloning into 'rag-rfb'...
remote: Enumerating objects: 523, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 523 (delta 8), reused 0 (delta 0), pack-reused 504 (from 1)
Receiving objects: 100% (523/523), 4.10 MiB | 10.13 MiB/s, done.
Resolving deltas: 100% (34/34), done.


In [ ]:
questions = json.load(open("rag-rfb/questions_QA_2024_v1.0.json", "r"))

In [ ]:
questions[0]

{'question_number': '001',
 'question_summary': 'OBRIGATORIEDADE',
 'question_text': 'Quem está obrigado a apresentar a Declaração de Ajuste Anual relativa ao exercício de 2024, ano-calendário de 2023?',
 'answer': ['Está obrigada a apresentar a Declaração de Ajuste Anual (DAA) referente ao exercício de 2024, a pessoa',
  'física residente no Brasil que, no ano-calendário de 2023:',
  '1 - recebeu rendimentos tributáveis, sujeitos ao ajuste na declaração, cuja soma foi superior a R$ 30.639,90',
  '(trinta mil, seiscentos e trinta e nove reais e noventa centavos);',
  '2 - recebeu rendimentos isentos, não tributáveis ou tributados exclusivamente na fonte, cuja soma foi superior',
  'a R$ 200.000,00 (duzentos mil reais);',
  '3 - obteve, em qualquer mês, ganho de capital na alienação de bens ou direitos sujeito à incidência do imposto;',
  '4 - realizou operações de alienação em bolsas de valores, de mercadorias, de futuros e assemelhadas:',
  'a) cuja soma foi superior a R$ 40.000,00 (q

In [ ]:
search_tool = json.load(open("rag-rfb/referred_legal_documents_QA_2024_v1.0.json", "r"))

In [ ]:
search_tool[10]

{'filename': 'Ato Declaratório Interpretativo SRF nº 14, de 4 de maio de 2004.txt',
 'filedata': 'NORMAS Contraste \ue88a \ue8ad Ato Declaratório Interpretativo SRF nº 14, de 04 de maio de 2004 (Publicado(a) no DOU de 05/05/2004, seção , página 7) Multivigente Vigente Original Relacional Dispõe sobre a tributação das atividades do sistema de locação conjunta de unidades imobiliárias denominado de pool hoteleiro. O SECRETÁRIO DA RECEITA FEDERAL, no uso da atribuição que lhe confere o inciso III do art. 209 do Regimento Interno da Secretaria da Receita Federal, aprovado pela Portaria MF nº 259, de 24 de agosto de 2001, tendo em vista o disposto nos arts. 148, 149, 254, 654, 662 e 666, do Decreto nº 3.000, de 26 de março de 1999 - Regulamento do Imposto de Renda (RIR, de 1999), o disposto no caput do art. 57 da Lei nº 8.981, de 20 de janeiro de 1995, com a redação dada pelo art. 1º da Lei nº 9.065, de 20 de junho de 1995, o disposto nos arts. 2º e 3º da Lei nº 9.718, de 27 de novembro de 

In [ ]:
question_answer = []
qtd = 0

for question in questions:
    if (qtd < 20):
      quest = []
      quest.append(question["question_text"])
      quest.append(' '.join(question["answer_cleaned"]))
      question_answer.append(quest)
      qtd += 1
    else:
      break


In [ ]:
question_answer[0][0]

'Quem está obrigado a apresentar a Declaração de Ajuste Anual relativa ao exercício de 2024, ano-calendário de 2023?'

In [ ]:
len(question_answer)

20

In [ ]:
len(search_tool)

478

### Segmentação de documentos

In [ ]:
%pip install -qU langchain-text-splitters

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 11.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.0 which is incompatible.
langchain 0.3.27 requires langchain-text-splitters<1.0.0,>=0.3.9, but you have langchain-text-splitters 1.0.0 which is incompatible.


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
def segmentation(document, chunk_size, chunk_overlap):

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=[" "])

    segmented_docs = []
    for text in document:
        chunks = text_splitter.split_text(text["filedata"])
        segmented_docs.extend(chunks)

    return segmented_docs

In [ ]:
chunks = segmentation(search_tool, 1000, 100)

In [ ]:
print(f"Chunks gerados: {len(chunks)}")

Chunks gerados: 17412


In [ ]:
len(chunks)

17412

### Indexação

In [ ]:
%pip install -qU langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.0 which is incompatible.
langchain 0.3.27 requires langchain-text-splitters<1.0.0,>=0.3.9, but you have langchain-text-splitters 1.0.0 which is incompatible.


In [ ]:
%pip install -qU langchain-huggingface sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.0/488.0 kB 23.9 MB/s eta 0:00:00


In [ ]:
%pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 30.7 MB/s eta 0:00:00


In [ ]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

In [ ]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

In [ ]:
from sentence_transformers import SentenceTransformer
import torch
import numpy as np

In [ ]:
def indexEmbedding(text_chunk, model="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2", use_gpu = True):

    device = "cuda" if torch.cuda.is_available() and use_gpu else "cpu"

    model = SentenceTransformer(model, device=device)
    embeddings = model.encode(
        text_chunk,
        batch_size=32,
        show_progress_bar=True,
        convert_to_numpy=True,
        normalize_embeddings=True
    )

    dimension = embeddings.shape[1]
    index = faiss.IndexFlatIP(dimension)

    # Adiciona todos embeddings de uma vez (não em loop)
    index.add(embeddings.astype(np.float32))

    return index, embeddings

In [ ]:
index, embeddings = indexEmbedding(chunks)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/545 [00:00<?, ?it/s]

### Dense Search

In [ ]:
from langchain_core.tools import tool


def dense_search(query, index, embeddings, chunks, top_k=2, model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"):
    # gera embedding da query
    model = SentenceTransformer(model_name)
    query_emb = model.encode([query], convert_to_numpy=True)

    # busca no FAISS
    similarity, indices = index.search(query_emb, top_k)

    # retorna os chunks correspondentes
    return [chunks[i] for i in indices[0]]

In [ ]:
resp = dense_search("Quem está obrigado a apresentar a Declaração de Ajuste Anual relativa ao exercício de 2024, ano-calendário de 2023?", index, embeddings, chunks, top_k=2)

In [ ]:
resp

['do disposto no caput, no momento da criação da nova declaração, serão consideradas as informações relativas ao contribuinte referentes ao exercício de 2024, ano-calendário de 2023, recebidas pela RFB por meio, dentre outros: I - da Declaração do Imposto sobre a Renda Retido na Fonte - Dirf; II - da Declaração de Serviços Médicos e de Saúde - Dmed; III - da Declaração de Informações sobre Atividades Imobiliárias - Dimob; IV - do Sistema de Recolhimento Mensal Obrigatório - Carnê-Leão; V - da e-Financeira; VI - da Declaração sobre Operações Imobiliárias - DOI; VII - da Declaração de Benefícios Fiscais - DBF; VIII - das informações relativas às operações realizadas com criptoativos a que se refere a Instrução Normativa RFB nº 1.888, de 3 de maio de 2019 ; ou IX - de informações obtidas por meio de convênios entre a RFB e entidades públicas ou privadas. § 2º A Declaração de Ajuste Anual Pré-Preenchida contém algumas informações relativas a rendimentos, deduções, bens e direitos e dívidas

### Memory

In [ ]:
from pathlib import Path

In [ ]:
from typing import List, Tuple, Dict, Any

In [ ]:
MEMORY_PATH = Path("memories.json")

In [ ]:
# verifica se o arquivo memories existet e lê seu conteudo
def load_memories() -> Dict[str, Any]:
    if MEMORY_PATH.exists():
        return json.loads(MEMORY_PATH.read_text(encoding="utf-8"))
    return {}


In [ ]:
def save_memories(memories: Dict[str, Any]):
    MEMORY_PATH.write_text(json.dumps(memories, ensure_ascii=False, indent=2), encoding="utf-8")

In [ ]:
memories = load_memories()

In [ ]:
# função chamada toda a vez que o que uma pergunta é feita
def append_memory(user_id: str, question: str, answer: str, context_chunks: List[Dict[str, Any]]):
    if user_id not in memories:
      memories[user_id] = []
    memories[user_id].append({
        "question": question,
        "answer": answer,
        "context": context_chunks,
    })
    save_memories(memories)

In [ ]:
!pip install -q python-dotenv

In [ ]:
!pip install -qU "langchain[openai]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 7.6 MB/s eta 0:00:00


In [ ]:
!pip install openai

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from langchain_community.chat_models import ChatOpenAI
from langchain.chat_models import init_chat_model
import getpass

In [ ]:
load_dotenv("/content/openai_api_key.env")

True

In [ ]:
if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-5-nano", model_provider="openai")

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])


In [ ]:
def build_prompt(question: str, tone: str, context_chunks: List[Dict[str, Any]], memory: List[Dict[str, Any]]) -> str:
    memory_section = "\n".join([f"Q: {m['question']}\nA: {m['answer']}" for m in (memory or [])[-5:]])
    retrieved = "\n\n".join(context_chunks)
    prompt = f"""
          Você é um assistente especialista em imposto de renda (Brasil). Responda objetivamente, sem inventar leis. Se não tiver certeza, deixe explícito que precisa checar.
          Tom solicitado: {tone}

          MEMÓRIA RELEVANTE:
          {memory_section}

          DOCUMENTOS RECUPERADOS:
          {retrieved}

          PERGUNTA:
          {question}

          Responda de forma clara, cite quando basear a resposta nos documentos recuperados (indique a fonte) e finalize com uma linha 'CONFIDENCE: <baixa|média|alta>' estimando confiança.
          """
    return prompt

In [ ]:
def writer_agent(question: str, tone: str, user_id: str = "default_user") -> Tuple[str, List[Dict[str, Any]]]:
    retrieved = dense_search(question, index, embeddings, chunks, top_k=2)

    user_memory = memories.get(user_id, [])

    prompt = build_prompt(question, tone, retrieved, user_memory)

    resp = client.chat.completions.create(
                    model="gpt-5-nano",
                    messages=[{"role": "user", "content": prompt}],
                )
    content = resp.choices[0].message.content.strip()

    append_memory(user_id,question, content, retrieved)


    return content, retrieved

In [ ]:
from typing import TypedDict, List

class AgentState(TypedDict):
    question: str
    tone: str
    retrieved: List[str]
    answer: str

In [ ]:
def search_node(state: AgentState) -> AgentState:
    retrieved = dense_search(state["question"], index, embeddings, chunks, top_k=2)
    state["retrieved"] = retrieved
    return state

In [ ]:
def writer_node(state: AgentState) -> AgentState:
    answer, _ = writer_agent(state["question"], state["tone"])
    state["answer"] = answer
    return state

In [ ]:
from langgraph.graph import StateGraph, END

In [ ]:
graph = StateGraph(AgentState)

graph.add_node("searcher", search_node)
graph.add_node("writer", writer_node)

graph.set_entry_point("searcher")
graph.add_edge("searcher", "writer")
graph.add_edge("writer", END)

multi_agent = graph.compile()

In [ ]:
def run_multi_agent(question: str, tone: str = "neutro"):
    input_state = {"question": question, "tone": tone}
    result = multi_agent.invoke(input_state)
    print("\n Resposta do agente redator:")
    print(result["answer"])
    return result["answer"]

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
from numpy.linalg import norm

# Inicializa o modelo de embeddings (o mesmo usado para FAISS)
embed_model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# Função para calcular similaridade coseno
def cosine_similarity(a, b):
    return np.dot(a, b) / (norm(a) * norm(b))

ragas_scores = []

for question, reference_answer in question_answer:
    generated_answer = run_multi_agent(question, tone="formal")

    ref_emb = embed_model.encode(reference_answer, convert_to_numpy=True)
    gen_emb = embed_model.encode(generated_answer, convert_to_numpy=True)

    score = cosine_similarity(ref_emb, gen_emb)

    ragas_scores.append(score)
    print(f"Pergunta: {question}")
    print(f"Score semântico RAGAS: {score:.3f}\n")

avg_score = np.mean(ragas_scores)
print(f"Score médio geral RAGAS: {avg_score:.3f}")



 Resposta do agente redator:
Resposta objetiva:

- Estão obrigadas a apresentar a Declaração de Ajuste Anual relativa ao exercício de 2024 (ano-calendário de 2023) as pessoas físicas residentes no Brasil que, em 31/12/2023, se enquadram em pelo menos uma das hipóteses de obrigatoriedade previstas pela legislação da Receita Federal para o IRPF 2024. Em linhas gerais, essas hipóteses envolvem:
  - rendimentos tributáveis sujeitos ao IR acima do limite de obrigatoriedade;
  - rendimentos isentos, não tributáveis ou tributados exclusivamente na fonte cuja soma seja relevante para a obrigatoriedade;
  - bens ou direitos de valor total acima de determinado limite;
  - ganho de capital na alienação de bens ou direitos;
  - operações em bolsa, mercadorias, futuros, derivativos ou criptoativos, entre outras situações com implicação de declaração;
  - outras hipóteses previstas pela norma vigente para 2024 (ex.: atividade rural com determinados patamares, etc.).

- Observação: os limites e as h